# easy_ViTPose demo
Be sure to enable GPU runtime

In [1]:
# !git clone https://github.com/JunkyByte/easy_ViTPose.git
!cd easy_ViTPose/ && pip install -r requirements.txt && pip install -e .
!pip install huggingface_hub 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for ultralytics==8.2.48 from https://files.pythonhosted.org/packages/4a/aa/e4cd1a17aa1a87c2466380145ddf8d0a440c577fdd7391d45e0cc1b149d2/ultralytics-8.2.48-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/41.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/41.2 kB ? eta -:--:--
     ------------------- -------------------- 20.5/41.2 kB ? eta -:--:--
     ---------------------------- --------- 30.7/41.2 kB 325.1 kB/s eta 0:00:01
     ---------------------------- --------- 30.7/41.2 kB 325.1 kB/s eta 0:00:01
     -------------------------------------- 41.2/41.2 kB 219.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/793.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/793.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/793.7 kB ? eta -:--:--
   --------------------------------------


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install -U ultralytics

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/fc/49/253e8d31a02064d0ff14a8e335d43ea6d60e89bc877c8bdc1fa8f76b5daa/ultralytics-8.3.28-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/881.2 kB ? eta -:--:--
   ---------------------------------------- 10.2/881.2 kB ? eta -:--:--
   ---------------------------------------- 10.2/881.2 kB ? eta -:--:--
   - ------------------------------------- 41.0/881.2 kB 326.8 kB/s eta 0:00:03
   ------ ------------------------------- 143.4/881.2 kB 847.9 kB/s eta 0:00:01
   ------- ------------------------------ 163.8/881.2 kB 893.0 kB/s eta 0:00:01
   ---------------- ----------------------- 368.6/881.2 kB 1.5 MB/s eta 0:00:01
   ------------------------- -------------- 563.2/881.2 kB 1.8 MB/s eta 0:00:01
   ------------------------------ --------- 675.8/881.2 kB 2.1 MB/s eta 0:00:01
   ---------------------


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Restart the runtime (runtime -> restart runtime) to update installed packages versions

# Download the models

In [3]:
#@title Choose model and run this cell

MODEL_SIZE = 'h'  #@param ['s', 'b', 'l', 'h']
YOLO_SIZE = 'n'  #@param ['s', 'n']
DATASET = 'apt36k'  #@param ['coco_25', 'coco', 'wholebody', 'mpii', 'aic', 'ap10k', 'apt36k']
ext = '.pth'
ext_yolo = '.pt'

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
!pip install scikit-image -q


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install filterpy -q


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
from easy_ViTPose import VitInference
# from huggingface_hub import hf_hub_download
# from google.colab import files
MODEL_TYPE = "torch"
YOLO_TYPE = "torch"
# REPO_ID = 'JunkyByte/easy_ViTPose'
# FILENAME = os.path.join(MODEL_TYPE, f'{DATASET}/vitpose-' + MODEL_SIZE + f'-{DATASET}') + ext
#FILENAME_YOLO = 'yolov8/yolov8' + YOLO_SIZE + ext_yolo

# print(f'Downloading model {REPO_ID}/{FILENAME}')
model_path = 'models/vitpose-h-apt36k.pth'
#yolo_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME_YOLO)

yolo_path = 'models/best_yolo11s_20ep_640_b4.pt'

# Run inference

In [20]:
model = VitInference(model_path, yolo_path, MODEL_SIZE,
                     dataset=DATASET, yolo_size=640, is_video=False)

In [21]:
model.yolo.names

{0: 'pig',
 1: 'wolf',
 2: 'fox',
 3: 'buffalo',
 4: 'moose',
 5: 'bear',
 6: 'hare',
 7: 'tiger',
 8: 'deer',
 9: 'bird',
 10: 'bobcat',
 11: 'leopard',
 12: 'goat',
 13: 'yellow-throated marten'}

In [22]:
# Function to perform IQA (using variance of Laplacian for sharpness)
def perform_iqa(cropped_img):
    gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
    variance = cv2.Laplacian(gray, cv2.CV_64F).var()
    threshold = 100  # Adjust based on your requirements
    return 1 if variance > threshold else 0  # Return 1 for good quality, 0 for poor quality

# Function to check keypoint conditions
def check_conditions(keypoints):
    if keypoints is None:
        logging.warning("No keypoints detected.")
        return False

    keypoints = np.array(keypoints)
    
    # Check if keypoints have the expected shape
    if keypoints.ndim != 2 or keypoints.shape[1] != 3:
        logging.warning(f"Unexpected keypoints shape: {keypoints.shape}")
        return False

    conf_threshold = 0.35  # Confidence threshold for specific checks

    # Define keypoint indices for each body part based on zero-based indexing
    torso_keypoints = [4, 5, 6, 7, 9, 10, 12, 13, 15, 16]
    head_keypoints = [1, 2, 3]
    leg_tail_keypoints = [5, 8, 11, 14, 17]

    # Ensure that the keypoints list has enough keypoints
    if keypoints.shape[0] <= max(torso_keypoints + head_keypoints + leg_tail_keypoints):
        logging.warning("Insufficient number of keypoints detected.")
        return False

    # Check overall visibility (at least 80% of all keypoints are within the frame)
    visible_keypoints_ratio = np.sum(~np.isnan(keypoints[:, 0:2])) / (keypoints.shape[0] * 2)
    if visible_keypoints_ratio < 0.8:
        logging.warning(f"Less than 80% of keypoints are visible in the frame: {visible_keypoints_ratio*100:.2f}%")
        return False

    # Check torso visibility (at least 20% of torso keypoints should be within the frame and above confidence threshold)
    visible_torso_points = sum(~np.isnan(keypoints[torso_keypoints, 0]))
    torso_visible = visible_torso_points >= len(torso_keypoints) * 0.2

    if not torso_visible:
        logging.warning("Torso is obscured by more than 80%.")
        return False

    # Check head visibility (any head keypoint with confidence above threshold)
    head_visible = any(keypoints[head_keypoints, 2] > conf_threshold)
    if not head_visible:
        logging.warning("No head keypoints visible.")
        return False

    # Check legs and tail visibility (at least one leg or tail keypoint should be above threshold)
    leg_tail_visible = any(keypoints[leg_tail_keypoints, 2] > conf_threshold)
    if not leg_tail_visible:
        logging.warning("No leg or tail keypoints visible for large animal.")
        return False


# Function to draw keypoints (optional, for visualization)
def draw_keypoints(img, keypoints, offset):
    x_offset, y_offset = offset
    for i, kp in enumerate(keypoints):
        x, y, conf = kp
        if conf > 0:
            cv2.circle(img, (int(x + x_offset), int(y + y_offset)), 3, (0, 255, 0), -1)
    return img


In [23]:
!pwd

"pwd" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [24]:
import os
import cv2
import pandas as pd
import csv
from tqdm import tqdm
import numpy as np
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Assuming perform_iqa and check_conditions are defined as previously discussed

# Path to the images folder
images_folder = 'train_data_minprirodi/images'

# Prepare a list to accumulate submission records
submission_records = []

# Verify that the folder exists
if not os.path.exists(images_folder):
    logging.error(f"Images folder not found at {images_folder}. Please check the path.")
    raise FileNotFoundError(f"Images folder not found at {images_folder}.")

# Process each image with a progress bar
image_files = os.listdir(images_folder)

for image_file in tqdm(image_files, desc="Processing Images"):
    image_path = os.path.join(images_folder, image_file)
    img = cv2.imread(image_path)

    if img is None:
        logging.warning(f"Unable to read image {image_file}. Skipping.")
        continue

    height, width = img.shape[:2]

    result = model.yolo.predict(img)[0]

    if len(result) == 0:
        logging.info(f"No detections in image {image_file}.")
        continue

    boxes = result.boxes  # Boxes object for bbox outputs

    # Iterate over each detected box
    for box in boxes:
        # Assuming box.xyxy is a list-like object with [x1, y1, x2, y2]
        x1, y1, x2, y2 = box.xyxy.tolist()[0]  # Convert to list of floats
        conf = box.conf.tolist()[0]  # Confidence score
        cls = int(box.cls.tolist()[0])  # Class label

        # Debugging: Print box details
        print(f"Box Details - x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}, conf: {conf}, cls: {cls}")

        bbox_width = x2 - x1
        bbox_height = y2 - y1

        # Normalize bbox coordinates to [0,1]
        x_center = (x1 + bbox_width / 2) / width
        y_center = (y1 + bbox_height / 2) / height
        norm_width = bbox_width / width
        norm_height = bbox_height / height
        bbox_string = f"{x_center},{y_center},{norm_width},{norm_height}"

        bbox_class = 1  # Initialize as good quality

        # Check if bbox is smaller than 128 pixels in width or height
        if bbox_width < 128 or bbox_height < 128:
            bbox_class = 0
            submission_records.append({
                'Name': image_file,
                'Bbox': bbox_string,
                'Class': bbox_class
            })
            continue

        # Crop the image inside the bbox
        x1_int, y1_int, x2_int, y2_int = map(int, [x1, y1, x2, y2])
        # Ensure the coordinates are within image boundaries
        x1_int = max(0, x1_int)
        y1_int = max(0, y1_int)
        x2_int = min(width, x2_int)
        y2_int = min(height, y2_int)

        cropped_img = img[y1_int:y2_int, x1_int:x2_int]

        # # Perform IQA
        # try:
        #     iqa_result = perform_iqa(cropped_img)
        #     bbox_class = iqa_result
        # except Exception as e:
        #     logging.error(f"Error performing IQA on {image_file}: {e}")
        #     bbox_class = 0

        # if bbox_class == 0:
        #     submission_records.append({
        #         'Name': image_file,
        #         'Bbox': bbox_string,
        #         'Class': bbox_class
        #     })
        #     continue

        # Run VitPose inference on the cropped image
        try:
            keypoints_dict = model.inference(cropped_img)
            # Ensure keypoints_dict is not empty
            if not keypoints_dict:
                logging.warning(f"No keypoints detected for {image_file}.")
                bbox_class = 0
                submission_records.append({
                    'Name': image_file,
                    'Bbox': bbox_string,
                    'Class': bbox_class
                })
                continue
        except Exception as e:
            logging.error(f"Error running VitPose on {image_file}: {e}")
            bbox_class = 0
            submission_records.append({
                'Name': image_file,
                'Bbox': bbox_string,
                'Class': bbox_class
            })
            continue

        # Iterate over each set of keypoints in the dictionary
        for id, keypoints in keypoints_dict.items():
            # Debugging: Print keypoints shape
            print(f"Keypoints Shape for {image_file}, ID {id}: {keypoints.shape}")

            # Check conditions on keypoints
            try:
                conditions_met = check_conditions(keypoints)
                if not conditions_met:
                    bbox_class = 0
            except Exception as e:
                logging.error(f"Error checking conditions for {image_file}, ID {id}: {e}")
                bbox_class = 0

            # Append the result to the submission records
            submission_records.append({
                'Name': image_file,
                'Bbox': bbox_string,
                'Class': bbox_class
            })

            # Optional: Visualize keypoints (uncomment if needed)
            # from google.colab.patches import cv2_imshow
            # img_with_keypoints = draw_keypoints(img.copy(), keypoints, (x1_int, y1_int))
            # cv2_imshow(img_with_keypoints[..., ::-1])  # Convert BGR to RGB for correct color display
            # cv2.waitKey(0)

# Cell 7: Save the Submission File
# Create the submission DataFrame from the accumulated records
submission = pd.DataFrame(submission_records, columns=['Name', 'Bbox', 'Class'])

# Save the submission DataFrame to a CSV file without adding extra quotes
submission.to_csv('submission.csv', index=False, quoting=csv.QUOTE_MINIMAL)

logging.info("Processing complete. Submission saved to 'submission.csv'.")

# Optional: Preview the submission DataFrame
print(submission.head())
print(submission['Class'].value_counts())


Processing Images:   0%|                                                                                                                                                                                    | 0/1586 [00:00<?, ?it/s]


0: 480x640 1 wolf, 3 deers, 96.9ms
Speed: 3.0ms preprocess, 96.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
Box Details - x1: 1095.5340576171875, y1: 539.2051391601562, x2: 1618.263916015625, y2: 992.0350952148438, conf: 0.9323976039886475, cls: 8


2024-11-09 19:15:59,031 - WARNING - No keypoints detected for 1001958.jpg.


Box Details - x1: 0.7443695068359375, y1: 676.5661010742188, x2: 367.007568359375, y2: 1129.4345703125, conf: 0.8874838948249817, cls: 8


2024-11-09 19:15:59,136 - WARNING - No keypoints detected for 1001958.jpg.


Box Details - x1: 0.7320556640625, y1: 961.1614990234375, x2: 130.96803283691406, y2: 1190.127197265625, conf: 0.7169173359870911, cls: 8


2024-11-09 19:15:59,325 - WARNING - No keypoints detected for 1001958.jpg.
2024-11-09 19:15:59,412 - WARNING - No keypoints detected for 1001958.jpg.
Processing Images:   0%|                                                                                                                                                                            | 1/1586 [00:00<19:59,  1.32it/s]2024-11-09 19:15:59,513 - INFO - No detections in image 1002155.jpg.


Box Details - x1: 435.4141845703125, y1: 1105.097900390625, x2: 932.902587890625, y2: 1367.274169921875, conf: 0.4539020359516144, cls: 1


Processing Images:   0%|▏                                                                                                                                                                           | 2/1586 [00:00<09:46,  2.70it/s]2024-11-09 19:15:59,608 - INFO - No detections in image 1003251.jpg.
2024-11-09 19:15:59,716 - INFO - No detections in image 1003657.jpg.
Processing Images:   0%|▌                                                                                                                                                                           | 5/1586 [00:01<04:34,  5.76it/s]2024-11-09 19:15:59,920 - INFO - No detections in image 1004371.jpg.
2024-11-09 19:16:00,035 - INFO - No detections in image 1004391.jpg.
Processing Images:   1%|█▏                                                                                                                                                                         | 11/1586 [00:01<03:10,  8.25it/s]2024-11-09 19:16:00,575 - INFO - No d

          Name                                               Bbox  Class
0  1001958.jpg  0.7067182223002116,0.5316806369357638,0.272255...      0
1  1001958.jpg  0.09576873381932577,0.6270835664537218,0.19076...      0
2  1001958.jpg  0.034296898047129314,0.7469752417670356,0.0678...      0
3  1001958.jpg  0.35633249282836915,0.8584625244140625,0.25910...      0
Class
0    4
Name: count, dtype: int64


In [13]:
def normalized_bbox_to_pixel(bbox, image_width, image_height):
    """
    Convert normalized bbox [x_center, y_center, width, height] to pixel coordinates [x1, y1, x2, y2].
    """
    x_center, y_center, width, height = map(float, bbox.split(','))
    x_center *= image_width
    y_center *= image_height
    width *= image_width
    height *= image_height

    x1 = x_center - width / 2
    y1 = y_center - height / 2
    x2 = x_center + width / 2
    y2 = y_center + height / 2

    return [x1, y1, x2, y2]

def calculate_iou(boxA, boxB):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.
    Each box is a list of four coordinates: [x1, y1, x2, y2].
    """
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interWidth = max(0, xB - xA)
    interHeight = max(0, yB - yA)
    interArea = interWidth * interHeight

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    iou = interArea / float(boxAArea + boxBArea - interArea) if (boxAArea + boxBArea - interArea) != 0 else 0

    return iou


In [14]:
# Paths to the ground truth and submission CSV files
ground_truth_path = 'train_data_minprirodi/annotation.csv'
submission_path = 'submission.csv'

# Load the CSV files
ground_truth_df = pd.read_csv(ground_truth_path)
submission_df = pd.read_csv(submission_path)

# Preview the data
print("Ground Truth Sample:")
print(ground_truth_df.head())

print("\nSubmission Sample:")
print(submission_df.head())


Ground Truth Sample:
          Name                                               Bbox  Class
0  1001958.jpg  0.7075520833333333,0.5319444444444444,0.282812...      1
1  1001958.jpg  0.09505208333333333,0.6305555555555555,0.19010...      0
2  1001958.jpg  0.031510416666666666,0.7434027777777777,0.0630...      0
3  1002155.jpg  0.8135416666666667,0.6976851851851852,0.371875...      0
4  1002155.jpg  0.3221354166666667,0.7939814814814815,0.477604...      0

Submission Sample:
          Name                                               Bbox  Class
0  1001958.jpg  0.7060381889343261,0.5277205361260309,0.292926...      0
1  1001958.jpg  0.08605960408846537,0.6263884014553494,0.17119...      0
2  1001958.jpg  0.35563847223917644,0.8848807864718967,0.25511...      0
3  1001958.jpg  0.03664730588595073,0.7358268737792969,0.06945...      0


In [15]:
# Group ground truth by image
gt_grouped = ground_truth_df.groupby('Name')

# Group submission by image
submission_grouped = submission_df.groupby('Name')

In [16]:
# Initialize scores
detector_score = 0
classifier_score = 0

# Counters for total objects
total_gt_objects = 0
total_submission_objects = 0

# Additional Counters for Detected Metrics
detector_tp = 0  # True Positives
detector_fp = 0  # False Positives
detector_fn = 0  # False Negatives

# Additional Counters for Classification Metrics
classifier_cc = 0  # Correct Classifications
classifier_ic = 0  # Incorrect Classifications
classifier_fpc = 0  # False Positive Classifications

# Iterate over each image in ground truth
for image_name, gt_rows in tqdm(gt_grouped, desc="Calculating Detector Quality"):
    # Load the image to get dimensions
    image_path = os.path.join(images_folder, image_name)
    img = cv2.imread(image_path)

    if img is None:
        logging.warning(f"Unable to read image {image_name} for evaluation. Skipping.")
        continue

    img_height, img_width = img.shape[:2]

    # Get ground truth bboxes
    gt_bboxes = gt_rows['Bbox'].tolist()
    gt_bboxes_pixel = [normalized_bbox_to_pixel(bbox, img_width, img_height) for bbox in gt_bboxes]

    # Get submission bboxes for this image
    if image_name in submission_grouped.groups:
        sub_rows = submission_grouped.get_group(image_name)
        sub_bboxes = sub_rows['Bbox'].tolist()
        sub_bboxes_pixel = [normalized_bbox_to_pixel(bbox, img_width, img_height) for bbox in sub_bboxes]
        sub_classes = sub_rows['Class'].tolist()
    else:
        sub_bboxes_pixel = []
        sub_classes = []

    total_gt_objects += len(gt_bboxes_pixel)
    total_submission_objects += len(sub_bboxes_pixel)

    # Keep track of matched ground truth
    matched_gt = set()

    # Iterate over submission bboxes and match with ground truth
    for sub_box in sub_bboxes_pixel:
        best_iou = 0
        best_gt_idx = -1
        for gt_idx, gt_box in enumerate(gt_bboxes_pixel):
            if gt_idx in matched_gt:
                continue
            iou = calculate_iou(sub_box, gt_box)
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = gt_idx
        if best_iou > 0.5:
            detector_score += 1  # Correct detection
            detector_tp += 1
            matched_gt.add(best_gt_idx)
        else:
            detector_score -= 1  # False positive
            detector_fp += 1

    # Penalize for missed ground truth objects
    missed_objects = len(gt_bboxes_pixel) - len(matched_gt)
    detector_score -= missed_objects
    detector_fn += missed_objects


Calculating Detector Quality: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1586/1586 [00:31<00:00, 49.69it/s]


In [17]:
# Reset matched_gt for classifier evaluation
for image_name, gt_rows in tqdm(gt_grouped, desc="Calculating Classifier Quality"):
    # Load the image to get dimensions
    image_path = os.path.join(images_folder, image_name)
    img = cv2.imread(image_path)

    if img is None:
        continue  # Already logged in previous loop

    img_height, img_width = img.shape[:2]

    # Get ground truth bboxes and classes
    gt_bboxes = gt_rows['Bbox'].tolist()
    gt_bboxes_pixel = [normalized_bbox_to_pixel(bbox, img_width, img_height) for bbox in gt_bboxes]
    gt_classes = gt_rows['Class'].tolist()

    # Get submission bboxes and classes for this image
    if image_name in submission_grouped.groups:
        sub_rows = submission_grouped.get_group(image_name)
        sub_bboxes = sub_rows['Bbox'].tolist()
        sub_bboxes_pixel = [normalized_bbox_to_pixel(bbox, img_width, img_height) for bbox in sub_bboxes]
        sub_classes = sub_rows['Class'].tolist()
    else:
        sub_bboxes_pixel = []
        sub_classes = []

    # Keep track of matched ground truth
    matched_gt = set()

    # Iterate over submission bboxes and match with ground truth
    for sub_box, sub_class in zip(sub_bboxes_pixel, sub_classes):
        best_iou = 0
        best_gt_idx = -1
        for gt_idx, gt_box in enumerate(gt_bboxes_pixel):
            if gt_idx in matched_gt:
                continue
            iou = calculate_iou(sub_box, gt_box)
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = gt_idx
        if best_iou > 0.5:
            # Correct detection
            matched_gt.add(best_gt_idx)
            # Check if class matches
            if sub_class == gt_classes[best_gt_idx]:
                classifier_score += 5  # Correct class
                classifier_cc += 1
            else:
                classifier_score -= 5  # Incorrect class
                classifier_ic += 1
        else:
            # False positive in classification
            classifier_score -= 5
            classifier_fpc += 1

    # Penalize for missed ground truth objects (no class to assign, only detector)
    # Already handled in detector_score


Calculating Classifier Quality: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1586/1586 [00:31<00:00, 49.65it/s]


In [18]:
# Calculate total possible scores
# Assuming M is the number of auxiliary objects (not defined in the original description)
# Here, we'll consider M = 0 for simplicity. Adjust accordingly if you have M.
M = 0
N = total_gt_objects  # Number of ground truth objects

# Maximum possible score
max_score = (M + N) * 6

# Sum of detector and classifier scores
X = detector_score + classifier_score

# Final result
final_result = X
if X > 0:
    final_result = X / max_score * 6
elif X <= 0:
    final_result = 0

# Ensure the final result is not negative
final_result = max(final_result, 0)

print(f"\n--- Итоговые Метрики ---")
print(f"Всего объектов в разметке (GT): {N}")
print(f"Всего обнаруженных объектов (Submission): {total_submission_objects}\n")

print(f"Детектор:")
print(f"  True Positives (TP): {detector_tp}")
print(f"  False Positives (FP): {detector_fp}")
print(f"  False Negatives (FN): {detector_fn}\n")

print(f"Классификатор:")
print(f"  Correct Classifications (CC): {classifier_cc}")
print(f"  Incorrect Classifications (IC): {classifier_ic}")
print(f"  False Positive Classifications (FPC): {classifier_fpc}\n")


print(f"Итоговый Балл: {final_result:.2f} из {max_score}")


--- Итоговые Метрики ---
Всего объектов в разметке (GT): 1985
Всего обнаруженных объектов (Submission): 4

Детектор:
  True Positives (TP): 3
  False Positives (FP): 1
  False Negatives (FN): 1982

Классификатор:
  Correct Classifications (CC): 2
  Incorrect Classifications (IC): 1
  False Positive Classifications (FPC): 1

Итоговый Балл: 0.00 из 11910
